# Full breakdown analysis on the leader in strike-outs

The objective of this guide is to show how the tables can be queried to get valuable information. We try here to get, per year, the name of the player with the most strike-outs (regular season and post-season included) with his actual number of strike-outs.

This query in an interesting one to break down as it combines two tables with different structures. The ```pitching``` table contains a line per player and per year for the regular season, meaning every stats of a player for a particular season is on a single line. the ```pitchingpost``` table, on the contrary, details the results per match, meaning that for a single post-season, we have several lines to sum up.

Let's start by querying the ```playerid```, ```yearid``` and ```so``` from the ```pitching``` table to get the number of strike-outs per player and per year. To avoid having too many rows, we limit the output to 5 rows:

```sql
SELECT playerid, yearid, so
FROM pitching
LIMIT 5;
```

Which gives this:

|	playerid	|	yearid	|	so	|
|		---		|	---		|	---	|
|	bechtge01	|	1871	|	1	|
|	brainas01	|	1871	|	13	|
|	fergubo01	|	1871	|	0	|
|	fishech01	|	1871	|	15	|
|	fleetfr01	|	1871	|	0	|

Let's now add the ```pitchingpost``` table following those steps:
1. Create a ```LEFT JOIN``` on the playerid columns, which are present in both tables. The ```LEFT JOIN``` here will return all lines present in the first table, ```pitching```, plus the matching lines in the ```pitchingpost```
2. Get the total number of strike-outs per player, by adding the column ```so``` from ```pitching``` to the sum of all strike-outs of the different matches in the post-seasons.
3. We want to make sure that the strike-outs of a post-season are added to the strike-outs of the related season (e.g.: the strike-outs in the 2019 post-season must be added to the strike-outs of the 2019 season, not to another season), so we add a ```WHERE``` clause and link the years of both tables
4. As we have an aggregate in the ```SELECT ``` clause, we have to group our results, so the ```GROUP BY``` clause will be added

```sql
SELECT pi.playerid, pi.yearid, (pi.so + SUM(po.so)) AS total_so
FROM pitching pi
LEFT JOIN pitchingpost po ON pi.playerid = po.playerid
WHERE pi.yearid = po.yearid
GROUP BY pi.playerid, pi.yearid, po.yearid,pi.so
LIMIT 5;
```

This gives us :

|	playerid	|	yearid	|	total_so	|
|		---		|		---	|	---	|
|	aasedo01	|	1979	|	102	|
|	abadfe01	|	2014	|	51	|
|	abbotgl01	|	1975	|	51	|
|	abbotpa01	|	2000	|	104	|
|	abbotpa01	|	2001	|	123	|

We have now the number of strike-outs for every player and every seasons they played. However, our goal is to identify and keep the best pitcher per season. This is done by ordering the results first per ```pi.yearid``` and then by ```total_so``` by using a ```ORDER BY``` clause. Then, we add a ```DISTINCT ON``` clause that we apply to ```pi.yearid```, which allows us to keep the best result per year :

```sql
SELECT DISTINCT ON (pi.yearid)
	pi.playerid, pi.yearid, (pi.so + SUM(po.so)) AS total_so
FROM pitching pi
LEFT JOIN pitchingpost po ON pi.playerid = po.playerid
WHERE pi.yearid = po.yearid
GROUP BY pi.playerid, pi.yearid, po.yearid,pi.so
ORDER BY pi.yearid DESC, total_so DESC
LIMIT 5;
```

As a result, we have:

|	playerid	|	yearid	|	total_so	|
|		---		|	---		| ---	|
|	colege01	|	2019	|	373	|
|	verlaju01	|	2018	|	307	|
|	salech01	|	2017	|	320	|
|	scherma01	|	2016	|	296	|
|	kershcl01	|	2015	|	320	|

Finally, in order to make the results a litlle bit easier to read, let's give the player their full name instead of their playerid. This info is located in the ```people``` table, so we just need another ```LEFT JOIN```. Let's also change the ```yearid``` column to ```season``` and ```total_so``` by ```total_strikeouts``` and then everything will be clear :

```sql
SELECT DISTINCT ON (pi.yearid)
	pe.namefirst||' '||pe.namelast AS pitcher_name, pi.yearid AS season, (pi.so + SUM(po.so)) AS total_strikeouts
FROM pitching pi
LEFT JOIN pitchingpost po ON pi.playerid = po.playerid
LEFT JOIN people pe ON pi.playerid = pe.playerid
WHERE pi.yearid = po.yearid
GROUP BY pitcher_name, pi.yearid, po.yearid,pi.so
ORDER BY pi.yearid DESC, total_strikeouts DESC
LIMIT 5;
```

And voilà!

|	pitcher_name	|	season	|	total_strikeouts	|
|		---			|	---		|	---	|
|	Gerrit Cole	|	2019	|	373	|
|	Justin Verlander	|	2018	|	307	|
|	Chris Sale	|	2017	|	320	|
|	Max Scherzer	|	2016	|	296	|
|	Clayton Kershaw	|	2015	|	320	|

For the full results, just delete the ```LIMIT``` clause.